# Test Valuation Engines

This notebook walks through:

1. Loading historical SOFR fixings
2. Building a simple two-index yield curve
3. Valuing various products (bullet, IBOR cashflow, OIS cashflow, futures, swaps)

---

## 1. Imports & Setup

In [1]:
import sys, os

# compute the parent of tests → FixedIncomeLib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

Added to sys.path: c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib


In [ ]:
import pandas as pd
from data import DataCollection, Data1D, build_yc_data_collection
from date import Date
from yield_curve import YieldCurve
from valuation import IndexManager
from valuation import ValuationEngineRegistry
from product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from builders import create_products_from_data1d
from builders import build_yc_calibration_basket
from product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor

NameError: name 'Iterable' is not defined

## 2. Load SOFR Fixings

In [ ]:
IndexManager.instance()

## 3. Build Dummy Yield Curve

In [ ]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-05-05 x 2025-08-05", 97.25],
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-08-05 x 2025-11-05", 97.40],
        ["RFR FUTURE", "SOFR-FUTURE-3M", "2025-11-05 x 2026-02-05", 97.55],

        ["RFR SWAP", "USD-SOFR-OIS", "1Y",  0.0450],
        ["RFR SWAP", "USD-SOFR-OIS", "2Y",  0.0425],
        ["RFR SWAP", "USD-SOFR-OIS", "5Y",  0.0370],
        ["RFR SWAP", "USD-SOFR-OIS", "10Y", 0.0350],
    ],
    columns=["DATA TYPE", "DATA CONVENTION", "AXIS", "VALUE"],
)

In [ ]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [ ]:
def createproductfromdata(value_date: str, data_objs):
    products = []
    for d in data_objs:  
        products.extend(create_products_from_data1d(value_date=value_date, data1d=d))
    return products

value_date = "2025-01-02"
all_products = createproductfromdata(value_date, data_objs)


In [ ]:
for i, p in enumerate(all_products, 1):
    if isinstance(p, ProductRfrFuture):
        print(f"\n[{i}] ProductRfrFuture")
        print(p.accept(RfrFutureVisitor()))
    elif isinstance(p, ProductOvernightSwap):
        print(f"\n[{i}] ProductOvernightSwap")
        print(p.accept(OvernightSwapVisitor()))
    else:
        print(f"\n[{i}] {type(p).__name__}")
        try:
            print(p.accept())
        except Exception:
            print(vars(p))



[1] ProductRfrFuture
         Attribute       Value
0     MaturityDate  2025-08-05
1    EffectiveDate  2025-05-05
2  TerminationDate  2025-08-05
3    AccrualFactor        0.25
4      Compounding    COMPOUND
5            Index     SOFR-1B
6           Strike      0.0275
7         Notional         1.0
8         Currency         USD
9      LongOrShort        LONG

[2] ProductRfrFuture
         Attribute       Value
0     MaturityDate  2025-11-05
1    EffectiveDate  2025-08-05
2  TerminationDate  2025-11-05
3    AccrualFactor        0.25
4      Compounding    COMPOUND
5            Index     SOFR-1B
6           Strike       0.026
7         Notional         1.0
8         Currency         USD
9      LongOrShort        LONG

[3] ProductRfrFuture
         Attribute       Value
0     MaturityDate  2026-02-05
1    EffectiveDate  2025-11-05
2  TerminationDate  2026-02-05
3    AccrualFactor        0.25
4      Compounding    COMPOUND
5            Index     SOFR-1B
6           Strike      0.0245
7   

In [ ]:
basket = build_yc_calibration_basket(value_date="2025-01-02", data_objs=data_objs)
for item in basket:
    prod = item.product
    quote = item.quote

print(f"Basket size: {len(basket)}")
for i, it in enumerate(basket, 1):
    p = it.product
    print(f"\n[{i}] {it.data_type}  {it.data_convention}  AXIS={it.axis}  QUOTE={it.quote}")
    if isinstance(p, ProductRfrFuture):
        print(p.accept(RfrFutureVisitor()))
    elif isinstance(p, ProductOvernightSwap):
        print(p.accept(OvernightSwapVisitor()))
    else:
        print(type(p).__name__)
        print(vars(p))

Basket size: 7

[1] RFR FUTURE  SOFR-FUTURE-3M  AXIS=('2025-05-05', '2025-08-05')  QUOTE=97.25
         Attribute       Value
0     MaturityDate  2025-08-05
1    EffectiveDate  2025-05-05
2  TerminationDate  2025-08-05
3    AccrualFactor        0.25
4      Compounding    COMPOUND
5            Index     SOFR-1B
6           Strike      0.0275
7         Notional         1.0
8         Currency         USD
9      LongOrShort        LONG

[2] RFR FUTURE  SOFR-FUTURE-3M  AXIS=('2025-08-05', '2025-11-05')  QUOTE=97.4
         Attribute       Value
0     MaturityDate  2025-11-05
1    EffectiveDate  2025-08-05
2  TerminationDate  2025-11-05
3    AccrualFactor        0.25
4      Compounding    COMPOUND
5            Index     SOFR-1B
6           Strike       0.026
7         Notional         1.0
8         Currency         USD
9      LongOrShort        LONG

[3] RFR FUTURE  SOFR-FUTURE-3M  AXIS=('2025-11-05', '2026-02-05')  QUOTE=97.55
         Attribute       Value
0     MaturityDate  2026-02-05
1 

In [ ]:
build_methods = [
    {
      'TARGET':             'SOFR-1B',
      'DATA_TYPE':          'ZERO_RATE',
      'DATA_CONVENTION':    'SOFR-1B',
      'INTERPOLATION_METHOD':'PIECEWISE_CONSTANT'
    },
    {
      'TARGET':             'USD-LIBOR-BBA-3M',
      'DATA_TYPE':          'ZERO_RATE',
      'DATA_CONVENTION':    'USD-LIBOR-BBA-3M',
      'INTERPOLATION_METHOD':'PIECEWISE_CONSTANT'
    },
]


In [ ]:
yc = YieldCurve("2025-06-27", dc, build_methods)
print("Curve components:", yc.components.keys())

KeyError: "No data for key ('zero_rate', 'SOFR-1B')"


## 4. Value Bullet Cashflow


In [ ]:
# Bullet cashflow PV
bullet = ProductBulletCashflow("2028-05-25", "USD", 1_000_000, "LONG")
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    bullet
)
ve.calculateValue()
print("Bullet CF PV:", ve.value)

Bullet CF PV: ['USD', np.float64(991474.0537980911)]


## 5. Value IBOR Cashflow

In [ ]:
ibor_cf = ProductIborCashflow(
    startDate="2025-06-27",
    endDate="2025-09-27",
    index="USD-LIBOR-BBA-3M",
    spread=0.0,
    notional=1_000_000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {}, ibor_cf)
ve.calculateValue()
print("Ibor CF PV:", ve.value)

Ibor CF PV: ['USD', np.float64(-1019.824662745844)]


## 6. Value OIS Cashflow

In [ ]:
ois_cf = ProductOvernightIndexCashflow(
    effectiveDate="2025-06-27",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    spread=0.0,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"valuation_date": Date("2025-07-01")},
    ois_cf
)
ve.calculateValue()
print("OIS CF PV:", ve.value)

OIS CF PV: ['USD', np.float64(561.0250872598588)]


## 7. Value Futures

In [ ]:
# IBOR Future PV
future = ProductFuture(
    effectiveDate="2025-06-27",
    index="USD-LIBOR-BBA-3M",
    strike=99.5,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {}, future)
ve.calculateValue()
print("Future PV:", ve.value)

Future PV: ['USD', np.float64(92070.13490166105)]


In [ ]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-06-27",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="AVERAGE",
    strike=99.7,
    notional=1_000_000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"valuation_date": Date("2025-07-01")},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

RFR Future PV: ['USD', np.float64(-243897.49127401217)]


## 8. Value a Portfolio

In [ ]:
# Simple portfolio of bullet + IBOR CF
portfolio = ProductPortfolio([bullet, ibor_cf], weights=[0.5, 0.5])
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B", "valuation_date": Date("2025-07-01")},
    portfolio
)
ve.calculateValue()
print("Portfolio PV:", ve.value)

Portfolio PV: ['USD', np.float64(990454.2291353453)]


## 9. Value Swaps (IBOR & OIS)

In [ ]:
# IBOR Swap
ibor_swap = ProductIborSwap(
    effectiveDate="2025-06-27",
    maturityDate="2026-06-27",
    frequency="6M",
    iborIndex="USD-LIBOR-BBA-3M",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_swap
)
ve.calculateValue()
print("IBOR Swap PV:", ve.value, "Par Rate:", ve.parRate())

IBOR Swap PV: ['USD', np.float64(-2000126.8763746223)] Par Rate: 32.41980827800543


In [ ]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-06-27",
    maturityDate="2026-06-27",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRate())

OIS Swap PV: ['USD', np.float64(1999949.24423121)] Par Rate: 31.084772494647435
